In [1]:
url = "ussoccer.com/News/Federation-Services/2009/06/University-Of-Miami-President-Donna-E-Shalala-Joins-Team-To-Bring-FIFA-World-Cup-To-United-States-In.aspx"
label = "good"

In [3]:
!pip3 install qrcode

In [4]:
import qrcode

# Create a QR code object with a larger size and higher error correction
qr = qrcode.QRCode(version=3, box_size=20, border=10, error_correction=qrcode.constants.ERROR_CORRECT_H)


# Add the data to the QR code object
qr.add_data(url)

# Make the QR code
qr.make(fit=True)

# Create an image from the QR code with a black fill color and white background
img = qr.make_image(fill_color="black", back_color="white")

# Save the QR code image
img.save("qr_code.png")

In [ ]:
# randomly choose 100 URLs from the dataset and transform them into images (from pahkare Dataset)

In [1]:
!pip3 install pandas

In [2]:
import pandas as pd

def load_dataset_pakhare():
    file="../data/pakhare_urls.csv"
    df = pd.read_csv(file)
    # count
    print("Good samples:" + str(df.groupby('Class').size()['good']))
    print("Bad samples:" + str(df.groupby('Class').size()['bad']))
    print(df.info(verbose=False))
    # check for null values
    bool_series = pd.isnull(df["URLs"])
    print(df[bool_series])
    return df

df = load_dataset_pakhare()

Good samples:344821
Bad samples:75643
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420464 entries, 0 to 420463
Columns: 2 entries, URLs to Class
dtypes: object(2)
memory usage: 6.4+ MB
None
Empty DataFrame
Columns: [URLs, Class]
Index: []


In [3]:
df_sample_10000_good = df[df['Class']=='good'].sample(n=10000)
df_sample_10000_bad = df[df['Class']=='bad'].sample(n=10000)
print(df_sample_10000_good)
print(df_sample_10000_bad)

                                                     URLs Class
275062  amazon.com/Compilation-Diane-Dufresne/dp/B0000...  good
169839                                    celtictalk.net/  good
295913                       myspace.com/frankspowerplant  good
179770  mmamania.com/2011/5/20/2180813/ufc-133-ivan-me...  good
50466   fr.worker-participation.eu/Systemes-nationaux/...  good
...                                                   ...   ...
45857                        ncra.ca/news/drumroll-please  good
242029  nytimes.com/1996/04/16/arts/ballet-review-swan...  good
15830                         americancasinothemovie.com/  good
42275   tomshardware.com/forum/262635-32-laptop-play-dvds  good
381555   showcase.awn.com/showgallery.php?cat=786&ppuser=  good

[10000 rows x 2 columns]
                                                     URLs Class
114640                          aberfoyledental.ca/5c80w4   bad
140443  gulsproductions.com/css/baked-seafood-pasta-al...   bad
102576        

In [4]:
import qrcode
# df must have 2 columns: Class and URLs
def generate_qr_codes(df, saving_path = '../data/qr_images/pakhare_dataset/qr_code_', version = 3, box_size =20, border = 10, error_correction = qrcode.constants.ERROR_CORRECT_H, fill_color='black', back_color = 'white'):
    for index, row in df.iterrows():
        url = row["URLs"]
        label = row["Class"]
        print(url, index)
        qr = qrcode.QRCode(version=version, box_size=box_size, border=border, error_correction=error_correction)
        qr.add_data(url)
        qr.make(fit=True)
        img = qr.make_image(fill_color=fill_color, back_color=back_color)
        img.save(saving_path+str(index)+"_"+label+".png")

# version = 3
# box_size = 20
# border = 10
# error_correction = qrcode.constants.ERROR_CORRECT_H
# fill_color = "black"
# back_color = "white"
generate_qr_codes(df_sample_10000_good)
generate_qr_codes(df_sample_10000_bad)


amazon.com/Compilation-Diane-Dufresne/dp/B00004VOK2 275062
celtictalk.net/ 169839
myspace.com/frankspowerplant 295913
mmamania.com/2011/5/20/2180813/ufc-133-ivan-menjivar-vs-nick-pace-fight-booked-for-aug-6-in 179770
fr.worker-participation.eu/Systemes-nationaux/Pays/France/Links 50466
ezdeals.com.au/canberra/ 219387
pdfgeni.com/book/husqvarna-viking-1250-service-manual-pdf.html 35006
youtube.com/watch?v=GzbfJpIrKuw 355254
elyrics.net/read/j/june-pointer-lyrics/little-boy-sweet-lyrics.html 96965
parcjeandrapeau.com/history.html 66232
123people.com/s/richard+hyde 298864
imdb.com/name/nm0163247/ 355825
webmii.asia/Result.aspx/Diana/Diaz 129301
facebook.com/people/Benjamin-Kos/100000830648888 35827
cloroxcoupon.info/ 319606
tch-nl.bookdirect.com/ 145529
nosorigines.qc.ca/genealogielistfirstname.aspx?ancestor=1&Family=Godbout_544&lng=en 316121
stubhub.com/oakland-raiders-tickets/ 181564
bestbuy.com/site/Ashlin+-+Slim+Multimedia+Storage+Tower+-+Black/9425204.p?id=1218102968575&skuId=9425204

Configure a simple CNN
for image classification + image resize

In [5]:
!pip3 install opencv-python

In [6]:
# image resize
folder_with_qr_codes = 'D:\IdeaProjects\malicious_links\data\qr_images\pakhare_dataset'

import cv2
import os
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append((img, filename))
    return images

# images = load_images_from_folder(folder_with_qr_codes)

In [24]:
filename = images[0][1]
size_pixels_1 = len(images[0][0][0])
size_pixels_2 = len(images[0][0])

print(filename, size_pixels_1, size_pixels_2)

qr_code_100695_bad.png 1540 1540


In [27]:
max_dim_for_qr_images = max([len(image[0]) for image in images])

In [7]:
from PIL import Image
import PIL
import os
import glob

folder_for_resized_images = 'D:\IdeaProjects\malicious_links\data\qr_images\pakhare_dataset\\resize_200_200'

def resize_images_for_folder_and_save(initial_folder, save_folder, resize_dimensions = (200, 200)):
    directory_files = os.listdir(initial_folder)
    multiple_images = [file for file in directory_files if 'qr_code' in file and file.endswith(('.png'))]
    print(multiple_images)
    for image in multiple_images:
        img = Image.open(folder_with_qr_codes + '\\'+ image)
        img.thumbnail(size=resize_dimensions)
        print(img)
        # We would run the command below to save the images:
        img.save(save_folder + '\\'+ image, optimize=True)


resize_images_for_folder_and_save(folder_with_qr_codes, folder_for_resized_images, (200,200))

['qr_code_100011_bad.png', 'qr_code_100037_good.png', 'qr_code_100077_good.png', 'qr_code_100087_bad.png', 'qr_code_100143_good.png', 'qr_code_100146_bad.png', 'qr_code_100209_good.png', 'qr_code_100215_good.png', 'qr_code_100217_good.png', 'qr_code_100252_bad.png', 'qr_code_10028_good.png', 'qr_code_100298_bad.png', 'qr_code_100308_good.png', 'qr_code_100321_bad.png', 'qr_code_10032_bad.png', 'qr_code_100361_good.png', 'qr_code_100362_good.png', 'qr_code_100374_bad.png', 'qr_code_100380_bad.png', 'qr_code_100404_bad.png', 'qr_code_100436_bad.png', 'qr_code_100453_good.png', 'qr_code_100471_bad.png', 'qr_code_100474_bad.png', 'qr_code_100477_bad.png', 'qr_code_100485_good.png', 'qr_code_100509_bad.png', 'qr_code_100531_bad.png', 'qr_code_100541_bad.png', 'qr_code_100542_good.png', 'qr_code_100572_bad.png', 'qr_code_100587_bad.png', 'qr_code_10060_good.png', 'qr_code_100626_good.png', 'qr_code_100692_good.png', 'qr_code_100695_bad.png', 'qr_code_100697_bad.png', 'qr_code_100705_good.png

In [2]:
# Configure simple CNN for qr code images classification
import pandas as pd
folder_for_resized_images = 'D:\IdeaProjects\malicious_links\data\qr_images\pakhare_dataset\\resize_200_200'
folder_with_input_images = folder_for_resized_images 

# Split data into training (80%), testing (20%) and validation (20 % from training)

images = load_images_from_folder(folder_with_input_images)

# print(images)
images_array = [image[0] for image in images]
filename_array = [image[1] for image in images]
label_array = ['bad' in image[1] for image in images]
id_array = [image[1].split('_')[2] for image in images]
# for bad = True
#  for legitimate = False
# print(images_array)
# print(filename_array)
# print(label_array)
# print(id_array)

# data = {'image': images_array, 'filename': filename_array, 'label': label_array, 'id': id_array}
data = {'image': images_array, 'label': label_array}
df = pd.DataFrame(data)

print(df)

                                                 image  label
0    [[[255, 255, 255], [255, 255, 255], [255, 255,...   True
1    [[[255, 255, 255], [255, 255, 255], [255, 255,...  False
2    [[[255, 255, 255], [255, 255, 255], [255, 255,...  False
3    [[[255, 255, 255], [255, 255, 255], [255, 255,...  False
4    [[[255, 255, 255], [255, 255, 255], [255, 255,...   True
..                                                 ...    ...
395  [[[255, 255, 255], [255, 255, 255], [255, 255,...  False
396  [[[255, 255, 255], [255, 255, 255], [255, 255,...  False
397  [[[255, 255, 255], [255, 255, 255], [255, 255,...   True
398  [[[255, 255, 255], [255, 255, 255], [255, 255,...   True
399  [[[255, 255, 255], [255, 255, 255], [255, 255,...  False

[400 rows x 2 columns]


In [3]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, stratify= df[['label']])

train_train_df, validation_df = train_test_split(train_df, test_size=0.2, stratify= train_df[['label']])

print(train_train_df, validation_df, test_df)


C:\Users\Ioana\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


                                                 image  label
299  [[[255, 255, 255], [255, 255, 255], [255, 255,...  False
152  [[[255, 255, 255], [255, 255, 255], [255, 255,...  False
100  [[[255, 255, 255], [255, 255, 255], [255, 255,...   True
363  [[[255, 255, 255], [255, 255, 255], [255, 255,...  False
199  [[[255, 255, 255], [255, 255, 255], [255, 255,...  False
..                                                 ...    ...
209  [[[255, 255, 255], [255, 255, 255], [255, 255,...  False
162  [[[255, 255, 255], [255, 255, 255], [255, 255,...  False
157  [[[255, 255, 255], [255, 255, 255], [255, 255,...  False
192  [[[255, 255, 255], [255, 255, 255], [255, 255,...   True
14   [[[255, 255, 255], [255, 255, 255], [255, 255,...  False

[256 rows x 2 columns]                                                  image  label
215  [[[255, 255, 255], [255, 255, 255], [255, 255,...  False
173  [[[255, 255, 255], [255, 255, 255], [255, 255,...   True
184  [[[255, 255, 255], [255, 255, 255], [255,

In [4]:
# Dataset: training (256 images) + validation (64 images) + testing (80 images)

print(train_train_df['label'].value_counts())
print(validation_df['label'].value_counts())
print(test_df['label'].value_counts())

False    128
True     128
Name: label, dtype: int64
False    32
True     32
Name: label, dtype: int64
True     40
False    40
Name: label, dtype: int64


In [5]:
!pip3 install tensorflow-gpu
# !pip3 install keras

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for tensorflow-gpu
Failed to build tensorflow-gpu


  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [18 lines of output]
  Traceback (most recent call last):
    File "<string>", line 2, in <module>
    File "<pip-setuptools-caller>", line 34, in <module>
    File "C:\Users\Ioana\AppData\Local\Temp\pip-install-qxeikr36\tensorflow-gpu_933543f6239042d1b7feee6276d831f3\setup.py", line 37, in <module>
      raise Exception(TF_REMOVAL_WARNING)
  Exception:
  
  The "tensorflow-gpu" package has been removed!
  
  Please install "tensorflow" instead.
  
  Other than the name, the two packages have been identical
  since TensorFlow 2.1, or roughly since Sep 2019. For more
  information, see: pypi.org/project/tensorflow-gpu
  
  
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tensorflow-gpu
ERROR: Could not build wheels for tensorflow-gpu, which is required to install pyproject.toml-

In [11]:
!pip3 install -U tensorflow

In [10]:
!pip3 install protobuf==3.20.* --user

In [6]:
import tensorflow as tf
from keras import layers, callbacks
from keras.models import Sequential

# Build CNN model
size_image = (200, 200)

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# 
model = Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape = [size_image[0], size_image[1],3]),
    layers.MaxPooling2D(),
    layers.Conv2D(64, (2, 2), activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, (2, 2), activation='relu'),
    layers.Flatten(),
    layers.Dense(100, activation='relu'),
    layers.Dense(2, activation ='softmax')
])

# print(model)
# 
model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics=['accuracy'])
# # tf.compat.v1.losses.sparse_softmax_cross_entropy
# # tf.losses.sparse_softmax_cross_entropy
callback = callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


Num GPUs Available:  0


In [12]:
print(train_train_df['image'].values)

[array([[[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],

        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],

        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],

        ...,

        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],

        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],

        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
        

In [16]:
import numpy as np


model.fit(x = np.asarray(train_train_df['image'].values), y=np.asarray(train_train_df['label'].values), epochs=2 , validation_data=(np.asarray(validation_df['image'].values), np.asarray(validation_df['label'].values)))

# model.fit(x = train_train_df['image'].values, y=train_train_df['label'].values, epochs=10 , validation_data=validation_df.values, callbacks=callback)

loss, accuracy = model.evaluate(test_df.values)
print("Loss: ", loss)
print("Accuracy: ", accuracy)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [1]:
import sys
print(sys.version)

3.10.1 (tags/v3.10.1:2cd268a, Dec  6 2021, 19:10:37) [MSC v.1929 64 bit (AMD64)]
